In [1]:
import pandas as pd
from cleaning import create_demand
import pyomo.environ as pyo
import pyomo

# Data import

In [2]:
nodes = pd.read_pickle("../../data/original/nodes.pkl")
channels = pd.read_pickle("../../data/original/channels.pkl")

## Modeling

In [3]:
pyomo.common.timing.report_timing()

In [4]:
model = pyo.ConcreteModel(name="Min cost flow problem")
model.NODES = pyo.Set(initialize=nodes.index)
model.CHANNELS = pyo.Set(initialize=[(channels.loc[i, "node1_pub"], channels.loc[i, "node2_pub"]) for i in channels.index])

           0 seconds to construct Block ConcreteModel; 1 index total
        0.06 seconds to construct Set NODES; 1 index total
        0.48 seconds to construct Set CHANNELS; 1 index total


In [5]:
nodes = create_demand(nodes, 500000)

Transaction of 500000 sats.
Sender: HighlighterSupermarket
Receiver: Odin 17.4 10:00.


In [6]:
model.x = pyo.Var(model.CHANNELS, domain=pyo.Binary)
model.a = pyo.Var(model.CHANNELS, domain=pyo.NonNegativeReals, bounds=(0, max(nodes["demand"])))

        0.28 seconds to construct Var x; 89898 indices total
        0.31 seconds to construct Var a; 89898 indices total


In [7]:
channels.reset_index(inplace=True)
channels.set_index(["node1_pub", "node2_pub"], inplace=True)
channels.sort_index(inplace=True)

In [8]:
def objective_function(model: pyo.ConcreteModel):
    return sum(model.a[i] * channels.loc[i, "rate_fee"] for i in model.CHANNELS) + sum(model.x[i] * channels.loc[i, "base_fee"] for i in model.CHANNELS)

model.totalCost = pyo.Objective(rule=objective_function(model), sense=pyo.minimize)

           0 seconds to construct Objective totalCost; 1 index total


### Constraints

#### Incoming channels rule

This constraint enforces the number of incoming channels for a node on the path.

$$
\sum_{(i,n) \in E} x_{i,n} = 1 \text{ } \forall n \in V \bigwedge i \ne s
$$

where $s$ is the source node, with a negative demand

In [9]:
#s = nodes[nodes["demand"] < 0].index.values
#d = nodes[nodes["demand"] > 0].index.values
#intermediaries = [(i, j) for i, j in channels.index if i != s and j != d]

In [10]:
def number_channels_rule(model: pyo.ConcreteModel, n):
    incoming = [model.x[(i, j)] for i, j in channels.index if j == n]
    outgoing = [model.x[(i, j)] for i, j in channels.index if i == n]
    return sum(incoming) == sum(outgoing)

model.NumberChannelsConstraint = pyo.Constraint(model.NODES, rule=number_channels_rule, name="Number channels constraint")

      407.25 seconds to construct Constraint NumberChannelsConstraint; 11984 indices total


In [11]:
#def incoming_channels_rule(model: pyo.ConcreteModel, n):
#    incoming = [model.x[(i, j)].value if j == n else 0 for i, j in intermediaries]
#    return sum(incoming) == 1
#
#model.IncomingChannelsConstraint = pyo.Constraint(model.NODES, rule=incoming_channels_rule, name="Incoming channels constraint")


#### Outgoing channels rule

This constraint enforces the number of outgoing channels for a node on the path.

$$
\sum_{(n,j) \in E} x_{n,j} = 1 \text{ } \forall n \in V \bigwedge j \ne d
$$

where $d$ is the destination node, with a positive demand

In [12]:
#def outgoing_channels_rule(model: pyo.ConcreteModel, n):
#    outgoing = [model.x[(i, j)].value if i == n else 0 for i, j in intermediaries]
#    return sum(outgoing) == 1
#
#model.OutgoingChannelsConstraint = pyo.Constraint(model.NODES, rule=outgoing_channels_rule, name="Outgoing channels constraint")

#### Capacity constraint

$$amount_{i,j} \le capacity_{i,j} \times x_{i,j} \text{ } \forall (i,j) \in E$$

In [13]:
def capacity_constraint(model: pyo.ConcreteModel, a, b):
    return model.a[(a, b)] <=  channels.loc[(a, b), "capacity"] * model.x[(a, b)]

model.CapacityConstraint = pyo.Constraint(model.CHANNELS, rule=capacity_constraint, name="Capacity constraint")

       12.54 seconds to construct Constraint CapacityConstraint; 89898 indices total


#### Flow balance constraint

$$\sum_{(s,i) \in E} amount_{s,i} - \sum_{(i,t) \in E} amount_{i,d} = b_i \text{ } \forall i \in V$$

where $s$ is the source node, $d$ is the destination node, $i$ is every intermediary node


In [14]:
channels.reset_index(inplace=True)
channels.set_index("channel_id", inplace=True)

def flow_balance_constraint(model: pyo.ConcreteModel, n: str):
    InFlow = sum(model.a[(channels.loc[a, "node1_pub"], channels.loc[a, "node2_pub"])] for a in nodes.loc[n, 'incoming_channels'])
    OutFlow = sum(model.a[(channels.loc[a, "node1_pub"], channels.loc[a, "node2_pub"])] for a in nodes.loc[n, 'outgoing_channels'])
    return  OutFlow + nodes.loc[n, "demand"] == InFlow

model.FlowBalanceConstraint = pyo.Constraint(model.NODES, rule=flow_balance_constraint, name="Flow balance constrain")

channels.reset_index(inplace=True)
channels.set_index(["node1_pub", "node2_pub"], inplace=True)
channels.sort_index(inplace=True) 

        8.03 seconds to construct Constraint FlowBalanceConstraint; 11984 indices total


## Solving the model

In [15]:
opt = pyo.SolverFactory('cbc')
results = opt.solve(model, tee=True)

if (results.solver.status == pyo.SolverStatus.ok) and (results.solver.termination_condition == pyo.TerminationCondition.optimal):
    print('\nOptimal solution found')
elif results.solver.termination_condition == pyo.TerminationCondition.feasible:
    print('\nFeasible but not proven optimal solution found')
elif results.solver.termination_condition == pyo.TerminationCondition.infeasible:
    raise Exception("The model is infeasible")
else:
    print('\nSolver Status: ',  results.solver.status)
    raise Exception(results.solver.status)

print('\nObject function value = ', model.Objective())


           0 seconds to construct Var ONE_VAR_CONSTANT; 1 index total
      [    5.85] Generated LP representation
Welcome to the CBC MILP Solver 
Version: 2.10.8 
Build Date: May  9 2022 

command line - /usr/bin/cbc -printingOptions all -import /tmp/tmpamx0x2r1.pyomo.lp -stat=1 -solve -solu /tmp/tmpamx0x2r1.pyomo.soln (default strategy 1)
Option for printingOptions changed from normal to all
Presolve 95188 (-18678) rows, 166432 (-13364) columns and 477128 (-61160) elements
Statistics for presolved model
Original problem has 89898 integers (89898 of which binary)
Presolved problem has 83216 integers (83216 of which binary)
==== 38072 zero objective 3879 different
==== absolute objective values 3879 different
==== for integers 31696 zero objective 1018 different
==== for integers absolute objective values 1018 different
===== end objective counts


Problem has 95188 rows, 166432 columns (128360 with objective) and 477128 elements
There are 10564 singletons with objective 
Column breakd

In [16]:
from decimal import Decimal
DF_channels = pd.DataFrame()
c = 0
for index, value in model.a.extract_values().items():
    if value != 0:
        DF_channels.loc[c, "source"] = index[0]
        DF_channels.loc[c, "destination"] = index[1]
        DF_channels.loc[c, "source-alias"] = nodes.loc[index[0], "alias"]
        DF_channels.loc[c, "destination-alias"] = nodes.loc[index[1], "alias"]
        DF_channels.loc[c, "capacity"] = channels.loc[index, "capacity"]
        DF_channels.loc[c, "amount"] = Decimal(value)
        DF_channels.loc[c, "base_fee"] = channels.loc[(index[0], index[1]), "base_fee"]
        DF_channels.loc[c, "rate_fee"] = channels.loc[(index[0], index[1]), "rate_fee"]
        c += 1

DF_channels_pos = DF_channels[DF_channels["amount"]!=0]
DF_channels_pos

,source,destination,source-alias,destination-alias,capacity,amount,base_fee,rate_fee
0,027fc4bbf73eeb128262948c8ad068ec243e569ead212f...,02cfdc6b60e5931d174a342b20b50d6a2a17c6e4ef8e07...,HighlighterSupermarket,Voltage-C2,500000.0,500000,1.000,0.000240
1,02cfdc6b60e5931d174a342b20b50d6a2a17c6e4ef8e07...,03a93b87bf9f052b8e862d51ebbac4ce5e97b5f4137563...,Voltage-C2,cyberdyne.sh,4000000.0,500000,0.100,0.000000
2,030751b547f0e59c40086bd6a2b5efbaaccfa850aa5af5...,02c551a80ba5758c845fc4d554a234e119082fa36f30d1...,RatPoison,Odin 17.4 10:00,1000000.0,500000,1.000,0.000001
3,03a93b87bf9f052b8e862d51ebbac4ce5e97b5f4137563...,030751b547f0e59c40086bd6a2b5efbaaccfa850aa5af5...,cyberdyne.sh,RatPoison,4000000.0,500000,0.001,0.000000


In [17]:
DF_channels[DF_channels["amount"]> DF_channels["capacity"]]

,source,destination,source-alias,destination-alias,capacity,amount,base_fee,rate_fee


In [18]:
DF_fixed = pd.DataFrame()
c = 0
for index, value in model.x.extract_values().items():
    if value != 0:
        DF_fixed.loc[c, "source"] = index[0]
        DF_fixed.loc[c, "destination"] = index[1]
        DF_fixed.loc[c, "used"] = Decimal(value)
        c += 1

DF_fixed_pos = DF_fixed[DF_fixed["used"]!=0]
DF_fixed_pos

,source,destination,used
0,02001828ca7eb8e44d4d78b5c1ea609cd3744be823c22c...,030b252eb8bb0f5e79af393d26dd5204f3c525af8b8404...,1
1,0201bfc5d9c14d50ebaf4dfd83d70df314ed2a9f0119f2...,0293952cbf1be801ba5d73ff34a950affb64fcbfe2d0f4...,1
2,0204edf87f5ee7d4e29a43756c87ff4f3152f3ea5de348...,02473eb084d0410351e343547c68dac1961d1c8e18d0da...,1
3,02055838411dba34c72329e9872fc66d2f0dc0292c049d...,0337d0c749f89d24e75427e1fec4ce2ed4f7d5a38c2196...,1
4,02057ee0163d5bc00dcc59adcc4a90670d7cef27db7493...,02f7912be4665093ec0cf0507f24a27ac57c3dca1f9e86...,1
...,...,...,...
1406,03fd8cc2d81e7f52c05b799372c2dc16120fe1723f3d8d...,02315fe3619ffdea2561bcacecada87b226723f471a59f...,1
1407,03fe824c5b3a0fb78e7f2844bfd052c8fe729b48ce1ffb...,030b252eb8bb0f5e79af393d26dd5204f3c525af8b8404...,1
1408,03fee917a3c0636f95d2529985a57ed185da42f35413f8...,03cde60a6323f7122d5178255766e38114b4722ede08f7...,1
1409,03ff3e6c4078f9bad09f27100366b1e226f55331a3a66f...,03c4b52dab7fe8ea9be955f16c8bb0c763c3eb2cbe1286...,1


In [19]:
intersection = DF_fixed_pos.merge(DF_channels_pos, on=["source", "destination"], how="outer")
intersection

,source,destination,used,source-alias,destination-alias,capacity,amount,base_fee,rate_fee
0,02001828ca7eb8e44d4d78b5c1ea609cd3744be823c22c...,030b252eb8bb0f5e79af393d26dd5204f3c525af8b8404...,1,NaN,NaN,NaN,NaN,NaN,NaN
1,0201bfc5d9c14d50ebaf4dfd83d70df314ed2a9f0119f2...,0293952cbf1be801ba5d73ff34a950affb64fcbfe2d0f4...,1,NaN,NaN,NaN,NaN,NaN,NaN
2,0204edf87f5ee7d4e29a43756c87ff4f3152f3ea5de348...,02473eb084d0410351e343547c68dac1961d1c8e18d0da...,1,NaN,NaN,NaN,NaN,NaN,NaN
3,02055838411dba34c72329e9872fc66d2f0dc0292c049d...,0337d0c749f89d24e75427e1fec4ce2ed4f7d5a38c2196...,1,NaN,NaN,NaN,NaN,NaN,NaN
4,02057ee0163d5bc00dcc59adcc4a90670d7cef27db7493...,02f7912be4665093ec0cf0507f24a27ac57c3dca1f9e86...,1,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
1406,03fd8cc2d81e7f52c05b799372c2dc16120fe1723f3d8d...,02315fe3619ffdea2561bcacecada87b226723f471a59f...,1,NaN,NaN,NaN,NaN,NaN,NaN
1407,03fe824c5b3a0fb78e7f2844bfd052c8fe729b48ce1ffb...,030b252eb8bb0f5e79af393d26dd5204f3c525af8b8404...,1,NaN,NaN,NaN,NaN,NaN,NaN
1408,03fee917a3c0636f95d2529985a57ed185da42f35413f8...,03cde60a6323f7122d5178255766e38114b4722ede08f7...,1,NaN,NaN,NaN,NaN,NaN,NaN
1409,03ff3e6c4078f9bad09f27100366b1e226f55331a3a66f...,03c4b52dab7fe8ea9be955f16c8bb0c763c3eb2cbe1286...,1,NaN,NaN,NaN,NaN,NaN,NaN
